In [ ]:
# Set the Pyspark enviroment variables
import os
os.environ['SPARK_HOME'] = "/Users/patricia/Apps/Spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter' # only if you use jupyter notebook
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook' # only if you use jupyter notebook
os.environ['PYSPARK_PYTHON'] = 'python'

In [14]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("DataFrame-SQL").getOrCreate()

24/11/07 12:11:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark

24/11/07 12:00:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Load Data into a DataFrame

In [15]:
# Load the synthetic data into a DataFrame
data_file_path = "./data/persons.csv"
df = spark.read.csv(data_file_path, header=True, inferSchema=True)

In [4]:
# Display schema of DataFrame
df.printSchema()

# Dsplay content of DataFrame
df.show(5)

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------+---+------+------+
|          name|age|gender|salary|
+--------------+---+------+------+
|      John Doe| 30|  Male| 50000|
|    Jane Smith| 25|Female| 45000|
| David Johnson| 35|  Male| 60000|
|   Emily Davis| 28|Female| 52000|
|Michael Wilson| 40|  Male| 75000|
+--------------+---+------+------+
only showing top 5 rows



### Register the DataFrame as a Temporary Table

In [17]:
# Register the DataFrame as a Temporary Table
df.createOrReplaceTempView("my_table")

### Perform SQL-like Queries

In [18]:
# Select all rows where age is greater than 25
result = spark.sql("SELECT * FROM my_table WHERE age > 25")
result.show()

+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|          John Doe| 30|  Male| 50000|
|     David Johnson| 35|  Male| 60000|
|       Emily Davis| 28|Female| 52000|
|    Michael Wilson| 40|  Male| 75000|
|       Sarah Brown| 32|Female| 58000|
|        Robert Lee| 29|  Male| 51000|
|       Lisa Garcia| 27|Female| 49000|
|    James Martinez| 38|  Male| 70000|
|Jennifer Rodriguez| 26|Female| 47000|
|  William Anderson| 33|  Male| 62000|
|   Karen Hernandez| 31|Female| 55000|
|Christopher Taylor| 37|  Male| 69000|
|     Matthew Davis| 36|  Male| 67000|
|    Patricia White| 29|Female| 50000|
|     Daniel Miller| 34|  Male| 64000|
| Elizabeth Jackson| 30|Female| 52000|
|     Joseph Harris| 28|  Male| 53000|
|      Linda Martin| 39|Female| 71000|
+------------------+---+------+------+



In [19]:
# Compute the average salary by gender
avg_salary_by_gender = spark.sql("SELECT gender, AVG(salary) as avg_salary FROM my_table GROUP BY gender")
avg_salary_by_gender.show()

+------+----------+
|gender|avg_salary|
+------+----------+
|Female|   52300.0|
|  Male|   62100.0|
+------+----------+



### Creating and managing temporary views.

In [20]:
# Create a temporary view
df.createOrReplaceTempView("people")

In [21]:
# Query the temporary view
result = spark.sql("SELECT * FROM people WHERE age > 25")

result.show()

+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|          John Doe| 30|  Male| 50000|
|     David Johnson| 35|  Male| 60000|
|       Emily Davis| 28|Female| 52000|
|    Michael Wilson| 40|  Male| 75000|
|       Sarah Brown| 32|Female| 58000|
|        Robert Lee| 29|  Male| 51000|
|       Lisa Garcia| 27|Female| 49000|
|    James Martinez| 38|  Male| 70000|
|Jennifer Rodriguez| 26|Female| 47000|
|  William Anderson| 33|  Male| 62000|
|   Karen Hernandez| 31|Female| 55000|
|Christopher Taylor| 37|  Male| 69000|
|     Matthew Davis| 36|  Male| 67000|
|    Patricia White| 29|Female| 50000|
|     Daniel Miller| 34|  Male| 64000|
| Elizabeth Jackson| 30|Female| 52000|
|     Joseph Harris| 28|  Male| 53000|
|      Linda Martin| 39|Female| 71000|
+------------------+---+------+------+



In [22]:
# Check if a temporary view exists
view_exists = spark.catalog.tableExists("people")
view_exists

True

In [23]:
# Drop a temporary view
spark.catalog.dropTempView("people")

True

In [24]:
# Check if a temporary view exists
view_exists = spark.catalog.tableExists("people")
view_exists

False

### Subquries

In [25]:
# Create DataFrames
employee_data = [
    (1, "John"), (2, "Alice"), (3, "Bob"), (4, "Emily"),
    (5, "David"), (6, "Sarah"), (7, "Michael"), (8, "Lisa"),
    (9, "William")
]
employees = spark.createDataFrame(employee_data, ["id", "name"])

salary_data = [
    ("HR", 1, 60000), ("HR", 2, 55000), ("HR", 3, 58000),
    ("IT", 4, 70000), ("IT", 5, 72000), ("IT", 6, 68000),
    ("Sales", 7, 75000), ("Sales", 8, 78000), ("Sales", 9, 77000)
]
salaries = spark.createDataFrame(salary_data, ["department", "id", "salary"])

employees.show()

salaries.show()

+---+-------+
| id|   name|
+---+-------+
|  1|   John|
|  2|  Alice|
|  3|    Bob|
|  4|  Emily|
|  5|  David|
|  6|  Sarah|
|  7|Michael|
|  8|   Lisa|
|  9|William|
+---+-------+

+----------+---+------+
|department| id|salary|
+----------+---+------+
|        HR|  1| 60000|
|        HR|  2| 55000|
|        HR|  3| 58000|
|        IT|  4| 70000|
|        IT|  5| 72000|
|        IT|  6| 68000|
|     Sales|  7| 75000|
|     Sales|  8| 78000|
|     Sales|  9| 77000|
+----------+---+------+



In [26]:
# Register as temporary views
employees.createOrReplaceTempView("employees")
salaries.createOrReplaceTempView("salaries")

In [27]:
# Subquery to find employees with salaries above average
result = spark.sql("""
    SELECT name
    FROM employees
    WHERE id IN (
        SELECT id
        FROM salaries
        WHERE salary > (SELECT AVG(salary) FROM salaries)
    )
""")

result.show()

+-------+
|   name|
+-------+
|  Emily|
|  David|
|Michael|
|   Lisa|
|William|
+-------+



### Window Function

In [28]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [29]:
employee_salary = spark.sql("""
    select  salaries.*, employees.name
    from salaries 
    left join employees on salaries.id = employees.id
""")

employee_salary.show()

+----------+---+------+-------+
|department| id|salary|   name|
+----------+---+------+-------+
|        HR|  1| 60000|   John|
|        HR|  2| 55000|  Alice|
|        HR|  3| 58000|    Bob|
|        IT|  4| 70000|  Emily|
|        IT|  5| 72000|  David|
|        IT|  6| 68000|  Sarah|
|     Sales|  7| 75000|Michael|
|     Sales|  9| 77000|William|
|     Sales|  8| 78000|   Lisa|
+----------+---+------+-------+



In [30]:
# Create a window specification
window_spec = Window.partitionBy("department").orderBy(F.desc("salary"))

In [ ]:
# Calculate the rank of employees within each department based on salary
employee_salary.withColumn("rank", F.rank().over(window_spec)).show()

In [11]:
# Stop the SparkSession
spark.stop()